In [127]:
import pandas as pd
import numpy as np
import re
import emoji

from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer


In [128]:
df1 = pd.read_csv("Tweet_Asterix.csv", sep = ";", usecols= ["created_at", "id", "text"])
print("Shape : ", df1.shape)
print("Colonnes : ", df1.columns)
df1.head()

Shape :  (1387, 3)
Colonnes :  Index(['created_at', 'id', 'text'], dtype='object')


,created_at,id,text
0,2023-02-08 13:46:53+00:00,1623317443903987713,Très bon retour spectateurs Allociné pour #ali...
1,2023-02-08 13:42:54+00:00,1623316442069233664,"J’vous jure c’est ni une aventure, ni une comé..."
2,2023-02-08 13:37:09+00:00,1623314995378356230,📚 Dans les librairies\nÀ découvrir dès aujourd...
3,2023-02-08 13:21:26+00:00,1623311040099262466,"@CarrefourFrance Bonjour, je participe pour te..."
4,2023-02-08 12:58:20+00:00,1623305227460919297,@CarrefourFrance #AsterixEtObelixLEmpireDuMili...


In [129]:
# We remove hours
def date_format(df):
    df["created_at"] = df["created_at"].str[:10]
    df = df.rename(columns = { "created_at" :"Date" })
    return df

df1 = date_format(df1)
df1.head()

,Date,id,text
0,2023-02-08,1623317443903987713,Très bon retour spectateurs Allociné pour #ali...
1,2023-02-08,1623316442069233664,"J’vous jure c’est ni une aventure, ni une comé..."
2,2023-02-08,1623314995378356230,📚 Dans les librairies\nÀ découvrir dès aujourd...
3,2023-02-08,1623311040099262466,"@CarrefourFrance Bonjour, je participe pour te..."
4,2023-02-08,1623305227460919297,@CarrefourFrance #AsterixEtObelixLEmpireDuMili...


In [130]:
#We check duplicata :

def check_duplicate(df):
    return df.duplicated().any()

check_duplicate(df1)

False

In [160]:
x = df1.iloc[1386,2]
x

"#AsterixEtObelixLEmpireDuMilieu\n#GuillaumeCanetEstMort\n#Urgence !!! Suite aux retours catastrophique des spectateurs et de la critique des premières séances #Cinema du nouveau film #Asterix. La production et les distributeurs viennent d'apprendre la mort de #GuillaumeCanet 😂 https://t.co/BHA8y5kTpo"

In [132]:
def text_processing(x):
    x = x.lower()
    x = re.sub(r"http\S+", "", x)
    x = re.sub(r"\U0001fae3", "", x)
    x = re.sub("@[A-Za-z0-9_]+","", x)
    x = re.sub("#[A-Za-z0-9_]+","", x)
    x = re.sub(r"-M-", "", x)
    x = re.sub(r"\n", " ", x)
    x = re.sub(r"&amp;", "et", x)
    x = re.sub(r"www.\S+", "", x)
    x = emoji.replace_emoji(x)
    return x

In [133]:
df1["clean_text"] = df1["text"].apply(text_processing)
df1.head()

,Date,id,text,clean_text
0,2023-02-08,1623317443903987713,Très bon retour spectateurs Allociné pour #ali...,très bon retour spectateurs allociné pour mal...
1,2023-02-08,1623316442069233664,"J’vous jure c’est ni une aventure, ni une comé...","j’vous jure c’est ni une aventure, ni une comé..."
2,2023-02-08,1623314995378356230,📚 Dans les librairies\nÀ découvrir dès aujourd...,dans les librairies à découvrir dès aujourd'h...
3,2023-02-08,1623311040099262466,"@CarrefourFrance Bonjour, je participe pour te...","bonjour, je participe pour tenter de gagner u..."
4,2023-02-08,1623305227460919297,@CarrefourFrance #AsterixEtObelixLEmpireDuMili...,merci !


In [156]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

print("Running Sentimental Analysis")

for i in df1.index:
    review = tb(df1.loc[i, "clean_text"]).sentiment[0]
    
    if (review > 0):
        df1.loc[i,"Sentiment"] = "Positive"
    elif (review < 0):
        df1.loc[i,"Sentiment"] = "Negative"
    else:
        df1.loc[i,"Sentiment"] = "Neutral"

df1

Running Sentimental Analysis


,Date,id,text,clean_text,Sentiment
0,2023-02-08,1623317443903987713,Très bon retour spectateurs Allociné pour #ali...,très bon retour spectateurs allociné pour mal...,Positive
1,2023-02-08,1623316442069233664,"J’vous jure c’est ni une aventure, ni une comé...","j’vous jure c’est ni une aventure, ni une comé...",Neutral
2,2023-02-08,1623314995378356230,📚 Dans les librairies\nÀ découvrir dès aujourd...,dans les librairies à découvrir dès aujourd'h...,Positive
3,2023-02-08,1623311040099262466,"@CarrefourFrance Bonjour, je participe pour te...","bonjour, je participe pour tenter de gagner u...",Positive
4,2023-02-08,1623305227460919297,@CarrefourFrance #AsterixEtObelixLEmpireDuMili...,merci !,Positive
...,...,...,...,...,...
1382,2023-02-01,1620790185536651264,⭐️ÉTOILES DE LA SEMAINE⭐️\n\n#AsterixEtObelixL...,étoiles de la semaine,Neutral
1383,2023-02-01,1620788985651134464,La bande originale d' #AstérixEtObélixLEmpireD...,la bande originale d' érixetobélixlempiredumil...,Positive
1384,2023-02-01,1620788185520549888,@SarahRempe @ctmieuxavant33 @destinationcine @...,je crois que c’était un reproche des ayant...,Positive
1385,2023-02-01,1620782556986896384,#Asterixetobelixlempiredumilieu j'ai tenu 40 m...,j'ai tenu 40 minutes devant cette daube ! per...,Negative


In [154]:
text = df1.loc[1382, ["text" ]]
text

text    ⭐️ÉTOILES DE LA SEMAINE⭐️\n\n#AsterixEtObelixL...
Name: 1382, dtype: object